https://github.com/NanoComp/meep/blob/master/python/examples/straight-waveguide.ipynb  

https://stackoverflow.com/questions/29454002/filenotfounderror-errno-2-no-such-file-or-directory-ffmpeg

3D可視化:  
https://meep.readthedocs.io/en/latest/Python_Tutorials/Basics/

In [1]:
import math
import meep as mp
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video

Using MPI version 3.1, 1 processes


In [2]:
Lwg = 60 # length of the PhCWG
Lbend = 20
Lstr =(Lwg-Lbend)/2
Tvideo=600
S = 1.5 # shift parameter "S", defines the waveguide width and the interface symmetry when Isym is set to 0
##### setting of parameters #####
dpml=1
widthPhC = 12+dpml
lengthPhC=Lwg
lengPhC_z=2*Lstr-Lbend/2
widthPhC_z=(20+Lbend/2)
Lwire = 10
ConnectionWaveguide = Lwire+dpml
r = 0.2572
rt=0.4
iud=1
pi=np.pi
n_eff = 2.65
fcen = 0.3 
df = 0.05
nfreq = 500 # number of frequencies at which to compute flux
resolution = 16
Isym = 0 # interface symmetry 0=free, 2=mirror, 1=glide
wgi = (S+3)/6
if Isym==0 :
    symi = (S+1)/4
elif Isym==1:
    symi = 1/2
elif Isym==2:
    symi = 0
p2hs=0
Si_w = 1 # silicon wire width
So_w = Si_w #source width
if S==-1:
    p2vs = 0.25 # vertical shift of port 2 for some waveguides like W1/6s&W1/3
    num_del = 10
elif S==-2:
    p2vs = 0.25 
    p2hs=0.5
    num_del = 5
else:
    p2vs = 0 
    num_del = 0


Nx = int(lengthPhC)
Ny = int(widthPhC)
eps = n_eff**2

length = lengthPhC + 2*ConnectionWaveguide
width = widthPhC
lengPhC_z=(2*Lstr-Lbend/2)
widthPhC_z=(20+Lbend/2)
Nx = int(lengthPhC)
Ny = int(widthPhC)
Nxz1=int(Lstr)
Nxz2=int(Lbend)
Nxz3=Nxz1
Nyz = 2*int(min([(lengPhC_z/2+Lbend/4)/np.sqrt(3),np.sqrt(3)*Lbend/2+20]))
eps = n_eff**2
xs=S/4
ys=S/4/np.sqrt(3) #modifying initial coordinate of the two PhC regions. Is zero if S=0


In [3]:
##### settings of geometry #####
# initialization

a = 400
c_const = 299792458

f_input=210 
fcen=1000*f_input*a/c_const


# initialization
cell = mp.Vector3(Lwg-1.5*Lbend+2*ConnectionWaveguide,(width+Lbend/2)*np.sqrt(3),0)
blk = mp.Block(mp.Vector3(Lwg-1.5*Lbend,(widthPhC+Lbend/2)*np.sqrt(3),mp.inf),
                    center=mp.Vector3(),
                    material=mp.Medium(epsilon=eps))
geometry=[blk]

for j in range(Nyz):
    for i in range(Nxz1+3+int(np.floor(1.5*j))):
        e11=xs+Lbend/4+3/4-i+1.5*j
        e12=ys+np.sqrt(3)*Lbend/4 + np.sqrt(3)/4+np.sqrt(3)*j/2
        vertices_u1=[mp.Vector3(e11+iud*rt*np.cos(pi/6+2*pi/3*0),e12+iud*rt*np.sin(pi/6+2*pi/3*0)),
                    mp.Vector3(e11+iud*rt*np.cos(pi/6+2*pi/3*1),e12+iud*rt*np.sin(pi/6+2*pi/3*1)),
                    mp.Vector3(e11+iud*rt*np.cos(pi/6+2*pi/3*2),e12+iud*rt*np.sin(pi/6+2*pi/3*2))
                    ]

        vertices_d1=[mp.Vector3(-e11+iud*rt*np.cos(pi/2+2*pi/3*0),-e12+iud*rt*np.sin(pi/2+2*pi/3*0)),
                    mp.Vector3(-e11+iud*rt*np.cos(pi/2+2*pi/3*1),-e12+iud*rt*np.sin(pi/2+2*pi/3*1)),
                    mp.Vector3(-e11+iud*rt*np.cos(pi/2+2*pi/3*2),-e12+iud*rt*np.sin(pi/2+2*pi/3*2))
                    ]
        geometry.append(mp.Prism(vertices=vertices_u1,height=mp.inf))
        geometry.append(mp.Prism(vertices=vertices_d1,height=mp.inf))

    for k in range(Nxz2):
        e21=xs+Lbend/4+3/4-k/2+1.5*j
        e22=ys+np.sqrt(3)*Lbend/4 + np.sqrt(3)/4-k*np.sqrt(3)/2+np.sqrt(3)*j/2
        vertices_u2=[mp.Vector3(e21+iud*rt*np.cos(pi/6+2*pi/3*0),e22+iud*rt*np.sin(pi/6+2*pi/3*0)),
                        mp.Vector3(e21+iud*rt*np.cos(pi/6+2*pi/3*1),e22+iud*rt*np.sin(pi/6+2*pi/3*1)),
                        mp.Vector3(e21+iud*rt*np.cos(pi/6+2*pi/3*2),e22+iud*rt*np.sin(pi/6+2*pi/3*2))
                    ]

        vertices_d2=[mp.Vector3(-e21+iud*rt*np.cos(pi/2+2*pi/3*0),-e22+iud*rt*np.sin(pi/2+2*pi/3*0)),
                        mp.Vector3(-e21+iud*rt*np.cos(pi/2+2*pi/3*1),-e22+iud*rt*np.sin(pi/2+2*pi/3*1)),
                        mp.Vector3(-e21+iud*rt*np.cos(pi/2+2*pi/3*2),-e22+iud*rt*np.sin(pi/2+2*pi/3*2))
                    ]
        geometry.append(mp.Prism(vertices=vertices_u2,height=mp.inf))
        geometry.append(mp.Prism(vertices=vertices_d2,height=mp.inf))
    for l in range(Nxz3):
        e31=xs-Lbend/4+3/4+l+1.5*j
        e32=ys-np.sqrt(3)*Lbend/4 + np.sqrt(3)/4+np.sqrt(3)*j/2
        vertices_u3=[mp.Vector3(e31+iud*rt*np.cos(pi/6+2*pi/3*0),e32+iud*rt*np.sin(pi/6+2*pi/3*0)),
                        mp.Vector3(e31+iud*rt*np.cos(pi/6+2*pi/3*1),e32+iud*rt*np.sin(pi/6+2*pi/3*1)),
                        mp.Vector3(e31+iud*rt*np.cos(pi/6+2*pi/3*2),e32+iud*rt*np.sin(pi/6+2*pi/3*2))
                    ]
        vertices_d3=[mp.Vector3(-e31+iud*rt*np.cos(pi/2+2*pi/3*0),-e32+iud*rt*np.sin(pi/2+2*pi/3*0)),
                        mp.Vector3(-e31+iud*rt*np.cos(pi/2+2*pi/3*1),-e32+iud*rt*np.sin(pi/2+2*pi/3*1)),
                        mp.Vector3(-e31+iud*rt*np.cos(pi/2+2*pi/3*2),-e32+iud*rt*np.sin(pi/2+2*pi/3*2))
                    ]
        geometry.append(mp.Prism(vertices=vertices_u3,height=mp.inf))
        geometry.append(mp.Prism(vertices=vertices_d3,height=mp.inf))

if num_del>0:
    for k in range(num_del):
        e01=xs+Lbend/4+3/4-(Nxz1-k)
        e02=ys+np.sqrt(3)*Lbend/4 + np.sqrt(3)/4
        vertices012=[mp.Vector3(e01+iud*rt*np.cos(pi/6+2*pi/3*0),e02+iud*rt*np.sin(pi/6+2*pi/3*0)),
                        mp.Vector3(e01+iud*rt*np.cos(pi/6+2*pi/3*1),e02+iud*rt*np.sin(pi/6+2*pi/3*1)),
                        mp.Vector3(e01+iud*rt*np.cos(pi/6+2*pi/3*2),e02+iud*rt*np.sin(pi/6+2*pi/3*2))
                    ]
        geometry.append(mp.Prism(vertices=vertices012,height=mp.inf,material=mp.Medium(epsilon=eps)))


        e03=-(xs+Lbend/4+3/4-(Nxz1-k)+p2hs)
        e04=ys-np.sqrt(3)*Lbend/4 + np.sqrt(3)/4
        vertices034=[mp.Vector3(e03+iud*rt*np.cos(pi/6+2*pi/3*0),e04+iud*rt*np.sin(pi/6+2*pi/3*0)),
                        mp.Vector3(e03+iud*rt*np.cos(pi/6+2*pi/3*1),e04+iud*rt*np.sin(pi/6+2*pi/3*1)),
                        mp.Vector3(e03+iud*rt*np.cos(pi/6+2*pi/3*2),e04+iud*rt*np.sin(pi/6+2*pi/3*2))
                    ]
        geometry.append(mp.Prism(vertices=vertices034,height=mp.inf,material=mp.Medium(epsilon=eps)))
# silicon wire
wire1 = mp.Block(mp.Vector3(ConnectionWaveguide,Si_w*np.sqrt(3),mp.inf),
                    center=mp.Vector3(-(2*Lstr-Lbend/2)/2-ConnectionWaveguide/2,p2vs*np.sqrt(3)+np.sqrt(3)*Lbend/4,0),
                    material=mp.Medium(epsilon=eps))
geometry.append(wire1)

wire2 = mp.Block(mp.Vector3(ConnectionWaveguide,Si_w*np.sqrt(3),mp.inf),
                    center=mp.Vector3((2*Lstr-Lbend/2)/2+ConnectionWaveguide/2,p2vs*np.sqrt(3)-np.sqrt(3)*Lbend/4,0),
                    material=mp.Medium(epsilon=eps))
geometry.append(wire2)


# Gaussian
sources = [mp.Source(mp.GaussianSource(fcen, fwidth=df),
                    component=mp.Hz,
                    center=mp.Vector3(-(2*Lstr-Lbend/2)/2-ConnectionWaveguide+dpml+Lwire/2,np.sqrt(3)*Lbend/4+p2vs*np.sqrt(3)),
                    size=mp.Vector3(0,So_w))]
            

# PML
pml_layers = [mp.PML(dpml)]

# z-symmetry (上下対称なら計算が軽くなる。対称性が無いなら消す)
# sym = [mp.Mirror(mp.Y, phase=-1)]


####
sim = mp.Simulation(cell_size=cell,
                    boundary_layers=pml_layers,
                    geometry=geometry,
                    sources=sources,
                    # symmetries=sym,
                    resolution=resolution)


f = plt.figure(dpi=150)
Animate = mp.Animate2D(sim, fields=mp.Hz, f=f,
        #field_parameters={'alpha':0.8, 'cmap':'RdBu', 'interpolation':'none'},
        realtime=False, normalize=True)

sim.run(mp.at_every(0.25,Animate),until=Tvideo) 

plt.close()

filename1 = 'S='+str(S)+'_z'+'freq='+str(f_input)+'_T='+str(Tvideo)
filename2 = '/mnt/d/dai_meep/tri/media/'+filename1
filename = filename2+".mp4"
fps = 60
Animate.to_mp4(fps,filename)
Video(filename)

-----------
Initializing structure...
time for choose_chunkdivision = 0.207353 s
Working in 2D dimensions.
Computational cell is 52 x 39.8125 x 0 with resolution 16
     block, center = (0,0,0)
          size (30,39.8372,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (7.0225,7.0225,7.0225)
     prism, center = (6.125,9.30977,5e+19)
          height 1e+20, axis (0,0,1), sidewall angle: 0 radians, 3 vertices:
          (6.47141,9.50977,0)
          (5.77859,9.50977,0)
          (6.125,8.90977,0)
          dielectric constant epsilon diagonal = (1,1,1)
     prism, center = (-6.125,-9.30977,5e+19)
          height 1e+20, axis (0,0,1), sidewall angle: 0 radians, 3 vertices:
          (-6.125,-8.90977,0)
          (-6.47141,-9.50977,0)
          (-5.77859,-9.50977,0)
          dielectric constant epsilon diagonal = (1,1,1)
     prism, center = (5.125,9.30977,5e+19)
          height 1e+20, axis (0,0,1), sidewall angle: 0 radians, 3 vertices:
 